# imports

In [75]:
import sys
# point this to appropriate snudda installation 
sys.path.append('/Applications/Blender.app/Contents/Resources/4.0/python/lib/python3.10/site-packages/Snudda') ##change me
# and to brainblenda location
sys.path.append('PATH_TO_BRAINBLENDA') ##change me
from brainblenda.brainblenda import bb_tools as bb

import bpy
import bmesh

import os
import glob
import h5py
import json

import mathutils
import math
import random
import numpy as np

from snudda.utils.snudda_path import snudda_parse_path, get_snudda_data
from snudda import SnuddaLoad

##point this to appropriate snudda data directory
snudda_data  = 'PATH_TO_SNUDDA_DATA' ##change me too

### uncomment to force reloads from within notebook without restarting the kernel
##import importlib
##importlib.reload(bb)

coord_conv = {'Mouselight': {'X':4, 'Y':3, 'Z':2}, 'Peng': {'X':2, 'Y':3, 'Z':4}}


### Setup blender space

In [101]:
bb.clear_scene()

[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor

### Draw a set of reconstructed neurons
By convention, these are centered at (0,0,0). This will create an array of morphologies and animate rotation about 'Y'.


In [100]:
morph_dir = 'example_data/neurons/'

lx = 0
ly = 0
offset = 600

for morph_path in glob.glob(os.path.join(morph_dir, '**', '*.swc')):
    name = morph_path.split('/')[-1].replace('.swc', '')
    bb.build_from_swc(filepath = morph_path, name = name, lx = lx, ly = ly, draw_axon = False, rotating = 'Y', rotation_time = 180)
    lx += offset
    if lx >= 6*offset:
        lx = 0
        ly += 1.5*offset

bb.frame_all()
bb.add_tracked_camera(target_name = '230123_20.001', altitude = 0, rotate = False, cam_scale = 6e3)

Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 2 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 2 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
I

### Import meshes from Allen
Requires .obj mesh files for Allen CCF regions. Can be downloaded via allenSDK. 
Calculate region volume while you're at it. 

OBS: Allen meshes are bilateral by default.

In [73]:
meshdir = 'example_data/Allen_Meshes/ccf'
regions = ['SNr', 'STN', 'GPe', 'CP', 'root']

colours = {'SNr': (52/255,168/255,224/255, 0.2),
           'STN': (232/255, 61/255, 61/255, 0.2), 
           'GPe': (161/255, 85/255, 168/255, 0.2), 
           'CP': (112/255, 182/255, 44/255, 0.2), 
           'root': (0, 0, 0, 0.01)}

for region in regions:
    obj = bb.import_allen_mesh(filepath = os.path.join(meshdir, f'{region}.obj'), colour = colours[region], alpha =colours[region][-1])
    print(f'{region} volume: {bb.get_mesh_volume(obj):.2f} mm\u00b3 ')
    
bb.frame_selected()

OBJ import of 'SNr.obj' took 15.8 ms
SNr volume: 1.50 mm³ 
OBJ import of 'STN.obj' took 2.8 ms
STN volume: 0.14 mm³ 
OBJ import of 'GPe.obj' took 16.4 ms
GPe volume: 1.46 mm³ 
OBJ import of 'CP.obj' took 54.5 ms
CP volume: 26.00 mm³ 
OBJ import of 'root.obj' took 157.2 ms
root volume: 516.35 mm³ 


### Draw a neuron from an external dataset
e.g. Peng, H., Xie, P., Liu, L. et al. Morphological diversity of single neurons in molecularly defined cell types. Nature 598, 174–181 (2021). https://doi.org/10.1038/s41586-021-03941-1

The morphologies in this dataset are *not* centered at (0,0,0). Therefore, location offsets can be ignored when drawing from the .swc file. 

OBS: drawing the axons for these neurons can be very slow, as the reconstructions are very detailed.

In [27]:
swc_path = 'example_data/Peng_2021/182725_2241_x9113_y11193.swc'
bb.build_from_swc(filepath = swc_path, name = 'example', coord_space = coord_conv['Peng'], draw_axon = False)


Info: Removed 58 vertice(s)


### Draw somas from a snudda simulation

In [14]:
network_path = 'example_data/networks/example_net/'
sl = SnuddaLoad(os.path.join(network_path, 'network-synapses.hdf5'))
neuron_coords = sl.data['neuron_positions']

print(f'Drawing {len(neuron_coords)} somas from simulation.')

coords_mesh = bb.draw_somas(neuron_coords, name = 'neuron coordinates', colour = (1,0,0.5,1), res = 2, scale_f = 1e6)
bb.flip_mesh_across_midline(coords_mesh) ## flip them to the other hemisphere, so we can also look at the full morphologies


Drawing 49 somas from simulation.


### Draw neuron morphologies from a snudda network

In [72]:
network_path = 'example_data/networks/example_net/'
sl = SnuddaLoad(os.path.join(network_path, 'network-synapses.hdf5'))
neurons = sl.data['neurons']
print(f'Drawing {len(neurons)} neurons from simulation.')

for neuron_id in range(len(neurons)): 
    obj = bb.draw_neuron_from_snudda(neurons = neurons, neuron_id = neuron_id, snudda_data = snudda_data, name = f'neuron_{neuron_id}')

bb.frame_all()

Drawing 49 neurons from simulation.


FileNotFoundError: [Errno 2] No such file or directory: '/Users/wst/Desktop/Karolinska/Simulation/Neuron/brainblenda/examples/PATH_TO_SNUDDA_DATA/networks/example_net/modified_morphologies/mSNrn_0-0.swc'

### Draw a neuron from a snudda network, with all of its postsynaptic partners

In [76]:
network_path = 'example_data/networks/example_net/'

post_ids, synapse_coords = bb.draw_presynaptic(network_path, pre_id = 0, snudda_data = snudda_data)


Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)
Info: Removed 0 vertice(s)


### Draw a neuron from a snudda network, with all of its presynaptic partners

In [74]:
network_path = 'example_data/networks/example_net/'

pre_ids, synapse_coords = bb.draw_postsynaptic(network_path, post_id = 23, snudda_data = snudda_data, match_synapses = True)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/wst/Desktop/Karolinska/Simulation/Neuron/brainblenda/examples/PATH_TO_SNUDDA_DATA/networks/example_net/modified_morphologies/lSNrn_0-23.swc'

### Draw simulation hypervoxels
OBS: this requires saving the hypervoxels coordinates when running Snudda detect

In [34]:
hv_coord_path = 'example_data/networks/example_net/hv_coords.json'

with open(hv_coord_path, 'r') as f:
    hv_coords = np.array(list(json.load(f).values()))
    
bb.draw_hypervoxels(hv_coords, alpha = 0.15, thickness = 15, scale_f = 1e6)

bpy.data.objects['hypervoxels']

### Add camera (rotating around the brain)

In [78]:
bb.add_tracked_camera(target_name = 'root', altitude = 0, rotate = True)

### Visualise projection densitites

In [86]:
import pandas as pd
proj_den = pd.read_csv('example_data/Allen_Connectivity/299895444_projection_density.csv')

sub = proj_den[(proj_den['3'] > 0.1)]  ##skip very low values
coords = sub[['0','1','2']].to_numpy()
values = sub['3']*0.5     # 0.5 factor for aesthetics

bb.voxels_from_coordinates_GN(coords, values = values)

bpy.data.objects['VoxelPoints.002']